# dev_utils

> TODO fill in description

In [ ]:
#| default_exp dev_utils

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from pathlib import Path
import pickle
import functools
import inspect
from typing import Callable

import fbdev
from fbdev._utils import get_function_from_py_file
from fbdev.complib import ExecComponent
from fbdev.runtime import BatchExecutor

In [ ]:
#|hide
show_doc(fbdev.dev_utils.method_from_py_file)

---

### method_from_py_file

>      method_from_py_file (file_path:Callable)

In [ ]:
#|export
def method_from_py_file(file_path:str):
    def decorator(orig_func):
        args = list(inspect.signature(orig_func).parameters.keys())
        is_async = inspect.iscoroutinefunction(orig_func)
        new_func = get_function_from_py_file(file_path, func_name=orig_func.__name__, args=args, is_async=is_async)
        if is_async:
            @functools.wraps(orig_func)
            async def wrapped_method(*args, **kwargs):
                await new_func(*args, **kwargs)
                await orig_func(*args, **kwargs)
        else:
            @functools.wraps(orig_func)
            def wrapped_method(*args, **kwargs):
                new_func(*args, **kwargs)
                orig_func(*args, **kwargs)
        return wrapped_method
    return decorator